In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys, os
from scipy.stats import zscore 

from rastermap import Rastermap, utils

# path to paper code
sys.path.insert(0, "/github/rastermap/paper")
import simulations, metrics, fig_splitting
from loaders import load_visual_data, load_alexnet_data

root = "/media/carsen/ssd2/rastermap_paper/"

os.makedirs(os.path.join(root, "simulations/"), exist_ok=True)

### 2D simulations

In [ ]:

filename = os.path.join(root, "simulations", "sim2D.npz")
if not os.path.exists(filename):
    # create simulated data with intrinsic dimensionality of 2
    simulations.make_2D_simulation(filename)

dat = np.load(filename)
spks = dat["spks"]
xi = dat["xi"]

### run algorithms
model = Rastermap(n_clusters=100, n_splits=0, n_PCs=400).fit(spks, normalize=False)
isort0 = model.isort 

model = Rastermap(n_clusters=100, n_splits=3, n_PCs=400).fit(spks, normalize=False)
isort_split = model.isort 
X_embedding = model.X_embedding

perplexities = [[10, 100], [10], [30], [100], [300]]
isorts_tsne = []
for i, perplexity in enumerate(perplexities):
    print(perplexity)
    y_tsne = metrics.run_TSNE(model.Usv, perplexities=perplexity, verbose=False)
    if i==0:
        isort_tsne = y_tsne[:,0].argsort()
    isorts_tsne.append(y_tsne[:,0].argsort())

isorts = [isort0, isort_split, *isorts_tsne]

### benchmark
knn_score, knn, rhos = simulations.benchmark_2D(xi, isorts)
        
np.savez(os.path.join(root, "simulations", "sim2D_results.npz"),
         X_embedding=X_embedding, isorts=np.array(isorts), 
         knn_score=knn_score, knn=knn, rhos=rhos, 
         xi=xi)

### visual cortex data

(this data will be shared upon publication)

In [ ]:
filename = os.path.join(root, "data/", "TX61_3x.npz")
stim_filename = os.path.join(root, "data/", "text5k_3x.mat")

out = load_visual_data(filename, stim_filename)
spks, istim, stim_times, xpos, ypos, run, ex_stim, img_pca, img_U, Ly, Lx = out

# run rastermap 
model = Rastermap(n_clusters=100, n_splits=3, nc_splits=25, locality=0.,
                             n_PCs=400, mean_time=False).fit(spks)
isort = model.isort
# bin by rastermap
n_neurons = len(isort)
n_bins = 500
bin_size = n_neurons // n_bins
X_embedding = zscore(utils.bin1d(spks[isort], bin_size, axis=0), axis=1)

# compute stimulus responses sresp and average over the three repeats
iss = np.zeros((3,5000), "int")
for j in range(5000):
    iss[:,j] = (istim==j).nonzero()[0][:3]
sresp = spks[:, stim_times]
sresp = sresp[:, iss].transpose((1,0,2))
snr_neurons = (zscore(sresp[0], axis=-1) * zscore(sresp[1], axis=-1)).mean(axis=1)

# bin rastermap by neurons
n_stim = sresp.shape[-1]
n_bins = 500
bin_size = n_neurons // n_bins
x = sresp[:, isort[:(n_neurons // bin_size) * bin_size]]
x = x.reshape(3, -1, bin_size, n_stim).mean(axis=2)
n_bins = x.shape[1]
snr = (zscore(x[0], axis=-1) * zscore(x[1], axis=-1)).mean(axis=-1)

# sort stimuli
model2 = Rastermap(n_clusters=100, n_splits=0, locality=0.,
                             n_PCs=400).fit(x.T)
isort2 = model2.isort

# mean over 3 repeats
sresp = sresp.mean(axis=0)
sresp = zscore(sresp, axis=1)
x = x.mean(axis=0)
x = zscore(x, axis=-1)

# ridge regression from 200 image PCs to 1000 rastermap components
itrain = np.arange(5000)%5>0
itest  = ~itrain

# ridge regression on training data with regularizer of 1e4
imgTimg = (img_pca[itrain].T @ img_pca[itrain])/itrain.sum()
imgTx   = (img_pca[itrain].T @ x[:, itrain].T)/itrain.sum()
B       = np.linalg.solve(imgTimg + 1e4 * np.eye(200), imgTx)

# reconstruct the receptive fields from the PCs
rfs = B.T @ img_U
rfs = np.reshape(rfs, (n_bins, Ly, Lx))

# evaluate model on test data
rpred = img_pca[itest] @ B
cpred = (zscore(rpred.T, 1) * zscore(x[:,itest], 1)).mean(1)

print(f"mean r on test data {cpred.mean()}")

np.savez(os.path.join(root, "results", "v1stimresp_proc.npz"),
         X_embedding=X_embedding, bin_size=bin_size, isort=isort, isort2=isort2, 
         xpos=xpos, ypos=ypos, x=x,
         stim_times=stim_times, run=run, ex_stim=ex_stim, rfs=rfs)

### alexnet activations to same images

(this data will be shared upon publication)

In [ ]:
filename = os.path.join(root, "data", "ann_fvs_Grayscale(224)_TX61_3X.npz")
sresp, ilayer, ipos, iconv, nmax = load_alexnet_data(filename)

# run rastermap
model = Rastermap(n_clusters=100, n_splits=3, nc_splits=25, locality=0.,
                             n_PCs=400, mean_time=False).fit(sresp)
isort = model.isort

# bin by rastermap
bin_size = 24
X_embedding = zscore(utils.bin1d(sresp[isort], bin_size, axis=0), axis=1)

# sort stimuli
model2 = Rastermap(n_clusters=100, n_splits=0, locality=0.,
                             n_PCs=400).fit(X_embedding.T)
isort2 = model2.isort

np.savez(os.path.join(root, "results", "alexnet_proc.npz"),
         X_embedding=X_embedding, bin_size=bin_size, isort=isort, isort2=isort2,
         ilayer=ilayer, ipos=ipos, iconv=iconv, nmax=nmax)

### make figures

In [ ]:
# root path has folders "simulations" and "results" with saved results
# will save figures to "figures" folder
fig = plt.figure(figsize=(14,8/3))
grid = plt.GridSpec(1,5, figure=fig, left=0.01, right=0.99, top=0.85, bottom=0.15, 
                    wspace = 0.15, hspace = 0.25)
il = 0
d = np.load(os.path.join(root, "simulations", "sim2D_results.npz"))
il = fig_splitting.panels_sim2d(fig, grid, il, **d)

fig.savefig(os.path.join(root, "figures", "suppfig_sim2D.pdf"), dpi=200)


In [ ]:
from scipy.io import loadmat

fig = plt.figure(figsize=(14, 10))
yratio = 14 / 10
grid = plt.GridSpec(4,5, figure=fig, left=0.01, right=0.99, top=0.94, bottom=0.07, 
                    wspace = 0.15, hspace = 0.25)
il = 0
areas = loadmat(os.path.join(root, "figures", "ctxOutlines.mat"), 
                squeeze_me=True)["coords"]
d = np.load(os.path.join(root, "results", "v1stimresp_proc.npz"))
il = fig_splitting.panels_v1stimresp(fig, grid, il, yratio, areas, **d, g0=0)

d = np.load(os.path.join(root, "results", "alexnet_proc.npz"))
il = fig_splitting.panels_alexnet(fig, grid, il, **d, g0=2)  

fig.savefig(os.path.join(root, "figures", "suppfig_visual.pdf"), dpi=200)
